In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
import geopandas

df = pd.read_csv("data/consolidation-etalab-schema-irve-statique-v-2.2.0-20230207.csv")

df_short= (
    df
    .loc[:,["id_station_local","nom_station", "code_insee_commune", "nbre_pdc", "gratuit", "consolidated_longitude", "consolidated_latitude"]]
)


geo_data = geopandas.read_file("data\communes-version-simplifiee.geojson")


df_short = (
    df
    .loc[:,["id_station_local","nom_station", "code_insee_commune", "nbre_pdc", "gratuit", "consolidated_longitude", "consolidated_latitude"]]
    .dropna(subset = ["nbre_pdc", "code_insee_commune"])
    .assign(code_insee_commune = lambda df_: df_.code_insee_commune.str.replace(".0","", regex = False))
    .groupby(by = "code_insee_commune")
    .agg({"nbre_pdc": np.sum, 
          "consolidated_longitude":"first",
          "consolidated_latitude":"first",
          })

    .reset_index()

)

df_shortGeometry = df_short.merge(
    geo_data,
    left_on = "code_insee_commune",
    right_on = "code"
).dropna(subset =["geometry"])

df_shortGeometry = geopandas.GeoDataFrame(df_shortGeometry)
df_shortGeometry ["logpdc"]=np.log(df_shortGeometry.nbre_pdc)

<<<<<<< HEAD

Carte de la france représantant le nombre de borne Electrique par commune


fig=px.choropleth_mapbox(df_shortGeometry,
                           geojson=df_shortGeometry.geometry,
                           locations=df_shortGeometry.index,
                           color="logpdc",
                           center={"lat": 45.5517, "lon": 5},
                           mapbox_style="open-street-map",
                           zoom=5,
                           color_continuous_scale=["wheat","burlywood","orangered","red","darkred","purple"],
                           hover_name = "nom",
                           hover_data=['nbre_pdc'],
                           title= "Carte de la france représantant le nombre de borne Electrique par commune",
                           labels={'nbre_pdc': 'Borne par ville'}
                           )
fig.update_layout(legend_title_text="sample data")
fig.show()
=======
# Gestion population 


In [ ]:
pop = pd.read_csv("data/pop/donnees_communes.csv", sep = ";")
pop

In [ ]:
meta_pop = pd.read_csv("data/pop/metadonnees_communes.csv", sep = ";")

meta_pop = (
  meta_pop
  .loc[meta_pop.cod_var == "COM",:]
)
meta_pop

In [ ]:
pop["code_commune"] = meta_pop.cod_mod.values
pop

In [ ]:
df_merged_pop = df_short.merge(
  pop,
  left_on = "code_insee_commune",
  right_on = "code_commune"
)

df_merged_pop["hab_par_pompe"] = df_merged_pop.PMUN / df_merged_pop.nbre_pdc


df_merged_popGeom = df_merged_pop.merge(
    geo_data,
    left_on = "code_insee_commune",
    right_on = "code"
)

# Gestion parc voiture


In [ ]:
df_parc = pd.read_csv("data/voitures-par-commune-par-energie.csv", sep = ";")

df_parc_parCommune = (
  df_parc
  .loc[df_parc.date_arrete == "2022-12-31",:]
  .groupby(by = "codgeo")
  .agg({"nb_vp_rechargeables_el": np.sum,
  	    "nb_vp_rechargeables_gaz": np.sum,
    	  "nb_vp": np.sum,
        "libgeo": "first"})
  .reset_index()
)
>>>>>>> merged_pop

## Analyse de voiture par habitant:

voiture


In [ ]:
def add_departements(df, insee_key):
  meta_depart = pd.read_csv('data/pop/metadonnees_departements.csv', sep = ";")
  meta_depart = (
    meta_depart
    .loc[meta_depart.lib_var == "Département",["cod_mod", "lib_mod"]]
  )

  df = (
    df
    .assign(code_dep = df[insee_key].str.slice(0,2))
  )

  df = df.merge(
    meta_depart,
    left_on = "code_dep",
    right_on = "cod_mod"
  )

  return df

# Grapĥiques


In [ ]:
nb_pdc_departement = (
  add_departements(df_short, "code_insee_commune")
  .groupby(by = "code_dep")
  .agg({"nbre_pdc": np.sum,
        "lib_mod": "first"}) 
  .reset_index()
  
)
nb_pdc_departement

In [ ]:
parc_dep = (
  add_departements(df_parc_parCommune, "codgeo")
  .groupby(by = "code_dep")
  .agg({"nb_vp": np.sum,
        "nb_vp_rechargeables_el": np.sum}) 
  .reset_index()
  

)
parc_dep

In [ ]:
pop_dep = pd.read_csv("data/pop/donnees_departements.csv", sep = ";")

pop_dep = pop_dep.loc[:,["CODDEP", "PMUN"]]
pop_dep

In [ ]:
px.scatter(nb_pdc_departement.merge(
                                  parc_dep,
                                  on = "code_dep"
                                ).merge(
                                  pop_dep,
                                  left_on = "code_dep",
                                  right_on = "CODDEP"
                                ),
x = "nb_vp_rechargeables_el",
y = "nbre_pdc",
hover_name = "lib_mod",
color = "PMUN",
trendline = "ols"

)

## Scatter departements
faire apparaitre les départements:
- 



# Carte intréactive

## Scatter: Stations de recharge


In [ ]:
px.scatter_mapbox(
  df,
  lat = "consolidated_latitude",
  lon = "consolidated_longitude",
  mapbox_style = "open-street-map",
  hover_name = "nom_station",
  center = go.layout.mapbox.Center(
            lat=45.18839381524741,
            lon=5.724469855557955
  ),
  zoom = 13
)

## Choropleth: Nombre de points de recharge par commune

TODO: Prendre le taux par habitant 


In [ ]:
px.choropleth_mapbox(
  df_shortGeometry,
  geojson=df_shortGeometry.geometry,
  locations=df_shortGeometry.index,
  color="nbre_pdc",
  center={"lat": 45.18839381524741, "lon": 5.724469855557955},
  mapbox_style="open-street-map",
  zoom=10,
  hover_name = "nom",

)

In [ ]:
to_choro = (
  geopandas.GeoDataFrame(df_merged_popGeom)
  .dropna(subset = "geometry")
  .loc[:,["geometry", "code_insee_commune", "nbre_pdc", "hab_par_pompe", "nom"]]
)
to_choro

In [ ]:
px.choropleth_mapbox(
  to_choro,
  geojson=to_choro.geometry,
  locations=to_choro.index,
  color="hab_par_pompe",
  center={"lat": 45.18839381524741, "lon": 5.724469855557955},
  mapbox_style="open-street-map",
  zoom=10,
  hover_name = "nom",

)